# SHAP values

- **Idea principale:** calcoliamo gli SHAP values per identificare quali sono i pixels (ovvero, i *kmers*) importanti ai fini della classificazione di un'immagine FCGR nel clade O oppure S.

- **Output sperato:** I *kmers* ritenuti importanti dal modelllo corrispondo ai kmers che spannano le varianti specifiche di ciascun COVID clade. Il clade S ha due marker variants (i.e. varianti caratteristiche del clade), quindi alla fine del notebook, speriamo di vedere che il modello considera questi kmers come importanti.

-------> Spiegazione facile sugli [shap values](https://readmedium.com/en/https:/towardsdatascience.com/shap-explained-the-way-i-wish-someone-explained-it-to-me-ab81cc69ef30)

## Imports

In [1]:
pip install shap

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import shap

from Bio import SeqIO

import pickle

import numpy as np
import os
import glob
from complexcgr import FCGR
import pandas as pd
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.layers import Input, Dense, Lambda, Dropout, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.models import Model
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image
import os
from keras.callbacks import ModelCheckpoint, CSVLogger

from tensorflow.keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from keras.regularizers import l2

2025-03-24 18:37:17.760362: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-24 18:37:18.162081: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-03-24 18:37:18.162114: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-03-24 18:37:18.200946: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 18:37:18.283135: I tensorflow/core/platform/cpu_feature_guar

## Extract kmers from COVID variants

### Load the reference

In [3]:
path_cv= r'/home/musimathicslab/Desktop/fcgr_snmg/covid-fcgr-classification-main/EPI_ISL_402124.fasta'
covid_reference = str(SeqIO.read(path_cv, "fasta").seq)


In [4]:
def reverse_complement(seq):
    """Restituisce il complemento inverso di una sequenza di DNA."""
    complement = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}
    return ''.join(complement[base] for base in reversed(seq))

def canonical_kmer(kmer):
    """Restituisce il k-mer canonico tra il k-mer originale e il suo complemento inverso."""
    rev_comp = reverse_complement(kmer)
    return min(kmer, rev_comp)

def get_canonical_kmers(seq,k=9):
    kmers = []
    for i in range(0,len(seq)-k+1):
        kmer = seq[i:i+k]
        kmers.append(canonical_kmer(kmer))
    return(kmers)   

def get_kmers(seq,k=9):
    kmers = []
    for i in range(0,len(seq)-k+1):
        kmer = seq[i:i+k]
        kmers.append(kmer)
        kmers.append(reverse_complement(kmer))
    return(kmers)   

In [5]:
covid_reference = str(SeqIO.read("/home/musimathicslab/Desktop/fcgr_snmg/covid-fcgr-classification-main/EPI_ISL_402124.fasta", "fasta").seq)

In [6]:
def get_kmers(seq,k=9):
    kmers = []
    for i in range(0,len(seq)-k+1):
        kmer = seq[i:i+k]
        kmers.append(kmer)
    return(kmers)   

In [ ]:
print(covid_reference[240]) #C
print(covid_reference[3036]) #C
print(covid_reference[23402]) #A
print(covid_reference[8781]) #C
print(covid_reference[11082]) #G
print(covid_reference[26143]) #G
print(covid_reference[28143]) #T

### Clade V

In [7]:
# Modifichiamlo il genoma di riferimento  
# sostituendo inserendo le marker variants per il clade V: G11083T,G26144T 1-based 
list_ref = list(covid_reference)
list_ref[11082] = 'T'
list_ref[26143] = 'T'
clade_V_ref = ''.join(list_ref)

In [8]:
# Aggiungiamo tutti i kmers spache spannano le marker mutations
k=9
clade_V_marker_kmers = []
clade_V_marker_kmers.extend(get_kmers(clade_V_ref[11082-k+1:11082+k]))
clade_V_marker_kmers.extend(get_kmers(covid_reference[11082-k+1:11082+k]))

clade_V_marker_kmers.extend(get_kmers(clade_V_ref[26143-k+1:26143+k]))
clade_V_marker_kmers.extend(get_kmers(covid_reference[26143-k+1:26143+k]))

In [9]:
clade_V_marker_kmers

['TTTTTTTTT',
 'TTTTTTTTT',
 'TTTTTTTTA',
 'TTTTTTTAT',
 'TTTTTTATG',
 'TTTTTATGA',
 'TTTTATGAA',
 'TTTATGAAA',
 'TTATGAAAA',
 'TTTTTTTTG',
 'TTTTTTTGT',
 'TTTTTTGTA',
 'TTTTTGTAT',
 'TTTTGTATG',
 'TTTGTATGA',
 'TTGTATGAA',
 'TGTATGAAA',
 'GTATGAAAA',
 'AATCGACGT',
 'ATCGACGTT',
 'TCGACGTTT',
 'CGACGTTTC',
 'GACGTTTCA',
 'ACGTTTCAT',
 'CGTTTCATC',
 'GTTTCATCC',
 'TTTCATCCG',
 'AATCGACGG',
 'ATCGACGGT',
 'TCGACGGTT',
 'CGACGGTTC',
 'GACGGTTCA',
 'ACGGTTCAT',
 'CGGTTCATC',
 'GGTTCATCC',
 'GTTCATCCG']

# Clade GK

In [10]:
# Modifichiamlo il genoma di riferimento  
# sostituendo inserendo le marker variants per il clade GK: C241T,C3037T,A23403G,C22995A 1-based 
list_ref = list(covid_reference)
list_ref[240] = 'T'
list_ref[3036] = 'T'
list_ref[23402] = 'G'
list_ref[22994] = 'A'
clade_GK_ref = ''.join(list_ref)

In [11]:
# Aggiungiamo tutti i kmers spache spannano le marker mutations
k=9
clade_GK_marker_kmers = []
clade_GK_marker_kmers.extend(get_kmers(clade_GK_ref[240-k+1:240+k]))
clade_GK_marker_kmers.extend(get_kmers(covid_reference[240-k+1:240+k]))

clade_GK_marker_kmers.extend(get_kmers(clade_GK_ref[3036-k+1:3036+k]))
clade_GK_marker_kmers.extend(get_kmers(covid_reference[3036-k+1:3036+k]))

clade_GK_marker_kmers.extend(get_kmers(clade_GK_ref[23402-k+1:23402+k]))
clade_GK_marker_kmers.extend(get_kmers(covid_reference[23402-k+1:23402+k]))

clade_GK_marker_kmers.extend(get_kmers(clade_GK_ref[22994-k+1:22994+k]))
clade_GK_marker_kmers.extend(get_kmers(covid_reference[22994-k+1:22994+k]))

In [ ]:
clade_GK_marker_kmers

In [12]:
# percorso dei file CSV
csv_file_path = '/home/musimathicslab/Desktop/fcgr_snmg/covid-fcgr-classification-main/dataset/fcgr/9Kmer/df_completo_GKV_9kmer.csv'
cladeV_path = '/home/musimathicslab/Desktop/fcgr_snmg/covid-fcgr-classification-main/dataset/fcgr/9Kmer/V/img'
cladeGK_path = '/home/musimathicslab/Desktop/fcgr_snmg/covid-fcgr-classification-main/dataset/fcgr/9Kmer/GK/img'


df = pd.read_csv(csv_file_path)

# Funzione per caricare le immagini e convertirle in array
def load_images(image_names, labels, cladeV_path, cladeGK_path):
    images = []
    for image_name, label in zip(image_names, labels):
        if label == 0:
            image_path = os.path.join(cladeGK_path, image_name)
        else:
            image_path = os.path.join(cladeV_path, image_name)
        image = load_img(image_path, color_mode="grayscale", target_size=(512, 512)) #Aggiunto color_mode per caricare l'immagine in scala di grigi
        image_array = img_to_array(image)
        images.append(image_array)
    return np.array(images)

# Carica le immagini
X = load_images(df['Image_Name'], df['Label'], cladeV_path, cladeGK_path)
y = df['Label'].values

In [13]:
def prepare_targets(y):
    
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(y)
    #print(integer_encoded)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    
    #print(onehot_encoded)
    #y_train_enc = onehot_encoded.transform(y_train)
    #y_test_enc = onehot_encoded.transform(y_test)
    return onehot_encoded

Y = prepare_targets(y)
print(Y)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]


In [14]:
# Dividere il dataset in train e test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
np.shape(X_train)
np.shape(y_train)

(7932, 2)

In [15]:
y_test_labels = np.argmax(y_test, axis=1) #[1,0] con la classe 0 mentre [0,1] con la classe 1

# Trova le etichette uniche
unique_labels = np.unique(y_test_labels)
print("Etichette uniche:", unique_labels)

# Conta le occorrenze delle etichette
counts = np.bincount(y_test_labels)
print("Frequenze delle etichette:", dict(zip(unique_labels, counts)))

Etichette uniche: [0 1]
Frequenze delle etichette: {0: 1042, 1: 941}


In [16]:
from tensorflow.keras.models import load_model

model = load_model('/home/musimathicslab/Desktop/fcgr_snmg/covid-fcgr-classification-main/dataset/fcgr/9Kmer/Model_GKV/trained_model3.h5')
model.summary()


2025-03-24 18:37:56.428306: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6821 MB memory:  -> device: 0, name: Quadro RTX 4000, pci bus id: 0000:b1:00.0, compute capability: 7.5


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 64, 64, 4)         260       
                                                                 
 batch_normalization (Batch  (None, 64, 64, 4)         16        
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 4)           1028      
                                                                 
 batch_normalization_1 (Bat  (None, 8, 8, 4)           16        
 chNormalization)                                                
                                                                 
 conv2d_2 (Conv2D)           (None, 1, 1, 4)           1028      
                                                                 
 batch_normalization_2 (Bat  (None, 1, 1, 4)           1

## Calculate SHAPs

Per ogni classe:
1. Calcoliamo le predictions del modello su tutte le immagini nel test set di quella classe
2. Calcoliamo gli SHAP su quelle predictions (gli SHAP saranno calcolati per ogni immagine) 
3. Facciamo la media del valore dello SHAP di ogni pixel per ogni immagine di quella classe
4. Traduciamo i pixels più importanti in *kmers*

In [ ]:
'''
# test np.mean
import numpy as np 
prova = [[[5,2,3],[1,2,3],[1,2,3]],[[1,2,3],[1,2,3],[1,2,3]]]
prova_array = np.array(prova)
avg_prova = np.abs(prova_array).mean(axis=0)
avg_prova
'''

In [17]:
np.random.seed(0)
#subset = np.random.choice(len(X_train), 1, replace=False)
#background, targets = [X_train[i] for i in subset], [y_train[i] for i in subset]
background, targets = X_train[np.random.choice(X_train.shape[0], 100, replace=False)], y_train[np.random.choice(len(y_train), 100, replace=False)]

In [18]:
print(len(background.shape))

4


In [19]:
print(len(targets.shape))

2


In [20]:
shap.explainers._deep.deep_tf.op_handlers["AddV2"] = shap.explainers._deep.deep_tf.passthrough
shap.explainers._deep.deep_tf.op_handlers["FusedBatchNormV3"] = shap.explainers._deep.deep_tf.passthrough
e = shap.DeepExplainer(model, background)

/home/musimathicslab/.local/lib/python3.11/site-packages/shap/explainers/_deep/deep_tf.py:99: UserWarning: Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.
  warnings.warn("Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode and some static graphs may not be supported. See PR #1483 for discussion.")
2025-03-24 18:37:56.984272: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


In [21]:
def top_n_indices(matrix, n):
    """
    Restituisce gli indici dei 'n' valori più alti nella matrice di forma (128, 128), in ordine decrescente.
    """
    # Appiattisci la matrice (trasforma in array 1D)
    flat_matrix = matrix.flatten()
    
    # Trova gli indici dei n valori più alti e ordina in ordine decrescente
    top_n_flat_indices = np.argsort(flat_matrix)[-n:][::-1]
    
    # Converti gli indici appiattiti in indici (riga, colonna) originali
    top_n_indices = np.unravel_index(top_n_flat_indices, matrix.shape)
    
    # Trasponi per ottenere una lista di tuple (riga, colonna)
    return list(zip(top_n_indices[0], top_n_indices[1]))

# trasformiamo le coordinate sull'array in kmer
k = 9
fcgr = FCGR(k)
coord2kmer = {(v[0]-1,v[1]-1): k for k,v in fcgr.kmer2pixel.items() }

### Clade V

In [22]:
# selezioniamo tutte le igms nel test set con clade S
x_test_V = np.array([img for img, y in zip(X_test, y_test) if y[0] == 1])

In [23]:
shap_values_all_V = e.shap_values(x_test_V)

/home/musimathicslab/.local/lib/python3.11/site-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(


In [24]:
shap_values_all_V.shape

(1042, 512, 512, 1, 2)

In [25]:
avg_shap_values_all_V = np.mean(shap_values_all_V[:, :, :, :, 1],axis=0).mean(axis=2)

In [ ]:
avg_shap_values_all_V.shape

In [50]:
avg_shap_values_all_V = np.mean(shap_values_all_V[:, :, :, :, 1],axis=0).mean(axis=2)

n = 200 
top_n_coords_V = top_n_indices(avg_shap_values_all_V,n)
top_n_kmers_V = [coord2kmer[(c[0],c[1])] for c in top_n_coords_V]
top_n_kmers_V

['TGGACCCTC',
 'CGTACCTTA',
 'CACACCCTC',
 'GCGACCCTC',
 'AGTTTACTC',
 'ATAACCTTA',
 'AAATCTTAG',
 'GCGACCTTA',
 'AAACACCTC',
 'TGCACCCTC',
 'CGTACCTAG',
 'TTTGCTCTC',
 'CGTGATCTC',
 'CGTGAACTC',
 'CGCACCACC',
 'ACCTTGTAG',
 'ACTTTACTC',
 'GGCTTACCG',
 'CATACCACC',
 'GGGACCACC',
 'ACTTCTTAG',
 'CTGCACTTA',
 'TGGACCACC',
 'CGCACCCTC',
 'CTACACACC',
 'GCTCGTCCG',
 'ATGTCTCTC',
 'GGCTCTTAG',
 'AACACCACC',
 'GTGACCCTC',
 'TCTCGTCGG',
 'GATGTGTTA',
 'TAAGTGTTA',
 'CTTACCACC',
 'AAATCAGCG',
 'TTTTGCCTC',
 'GATTCTTTA',
 'ATAACCTAG',
 'GGTGGAACC',
 'CTGTTTACC',
 'TTTCCCTTA',
 'CTGCACGCG',
 'ACCTCTTAG',
 'ACTGGAACC',
 'TCTACCATC',
 'GGTTCTTTA',
 'TAAACCTAG',
 'GATTTACTC',
 'GGCGGAACC',
 'ATAACCCTC',
 'CGTTTACTC',
 'GCTGTTATG',
 'CGTGGAACC',
 'CACACCACC',
 'GATTCTTAG',
 'CTATTACTC',
 'AGTTTTTAG',
 'GCTGCTTGT',
 'TCAACCTAG',
 'GCTCGTCGG',
 'TAAGAAACC',
 'AGTGTGGCG',
 'CGTACCTGT',
 'CGTGTGGCG',
 'TTTCAGTAG',
 'ATTTAAACC',
 'ATTACCAGA',
 'CTGTTAATG',
 'CATCACTTA',
 'AAATTTGTC',
 'GCTTCTTAG',
 'GATA

In [51]:
k2p = fcgr.kmer2pixel
for i in top_n_kmers_V:
    coords = k2p[i]
    print(avg_shap_values_all_V[coords[0]-1][coords[1]-1])

0.009016899699372603
0.005893341331548275
0.005654037937808169
0.005408370220104604
0.0047180139207304884
0.0037052528212728354
0.0035239945695917456
0.0033975646694737347
0.003151475211253187
0.003086872425331643
0.0029314679361849886
0.002803995364190441
0.0027275354523453206
0.002704660528538424
0.002641573618453838
0.002543734631180717
0.0024907306639050885
0.0024585887683276973
0.0024290058374212297
0.002414836636161595
0.002402054381035705
0.0024003690072942198
0.00238626984580348
0.0023202265585632368
0.002256685600357064
0.0022385782102344563
0.0022124592417041324
0.0021782582916226416
0.0021681732497259745
0.002162926032328232
0.0021003230198607167
0.0020356015269596796
0.002025006819491869
0.002013982199873166
0.0019958831772484507
0.0019497474946889232
0.0019386286501304835
0.0019353680629535684
0.0019232301449160951
0.0019192140260781557
0.0018981247502248067
0.0018976566806523177
0.0018858769012773675
0.0018515571388754513
0.001848814785770429
0.001803175606252754
0.001758

In [52]:
tot = 0 
for i in top_n_kmers_V:
    if i in clade_V_marker_kmers:
        print(f"Kmer: {i}")
        tot += 1
print(tot)

0


In [53]:
# Aggiungiamo tutti i kmers spache spannano le marker mutations
k=9
clade_V_marker_kmers1c = []
clade_V_marker_kmers2c = []
clade_V_marker_kmers1r = []
clade_V_marker_kmers2r = []
clade_V_marker_kmers1c.extend(get_kmers(clade_V_ref[11082-k+1:11082+k]))
clade_V_marker_kmers1r.extend(get_kmers(covid_reference[11082-k+1:11082+k]))

clade_V_marker_kmers2c.extend(get_kmers(clade_V_ref[26143-k+1:26143+k]))
clade_V_marker_kmers2r.extend(get_kmers(covid_reference[26143-k+1:26143+k]))


for i in top_n_kmers_V:
    if i in clade_V_marker_kmers1c:
        print(f"Clade: {i} presente in Clade_V_Marker_Kmers1c: {clade_V_marker_kmers1c}")


for i in top_n_kmers_V:
    if i in clade_V_marker_kmers1r:
        print(f"Clade: {i} presente in Clade_V_Marker_Kmers1r: {clade_V_marker_kmers1r}")
        


for i in top_n_kmers_V:
    if i in clade_V_marker_kmers2c:
        print(f"Clade: {i} presente in Clade_V_Marker_Kmers2c: {clade_V_marker_kmers2c}")


for i in top_n_kmers_V:
    if i in clade_V_marker_kmers2r:
        print(f"Clade: {i} presente in Clade_V_Marker_Kmers2r: {clade_V_marker_kmers2r}")

# Clade GK

In [54]:
# selezioniamo tutte le igms nel test set con clade GK
x_test_GK = np.array([img for img, y in zip(X_test, y_test) if y[0] == 0])

In [55]:
shap_values_all_GK = e.shap_values(x_test_GK)

In [56]:
shap_values_all_GK.shape

(941, 512, 512, 1, 2)

In [57]:
avg_shap_values_all_GK = np.mean(shap_values_all_GK[:, :, :, :, 1],axis=0).mean(axis=2)

In [58]:
avg_shap_values_all_GK.shape

(512, 512)

In [77]:
avg_shap_values_all_GK = np.mean(shap_values_all_GK[:, :, :, :, 1],axis=0).mean(axis=2)

n = 200 
top_n_coords_GK = top_n_indices(avg_shap_values_all_GK,n)
top_n_kmers_GK = [coord2kmer[(c[0],c[1])] for c in top_n_coords_GK]
top_n_kmers_GK

['TTTACCCTC',
 'TCTACCCTC',
 'TTTCGTCCG',
 'TAAACCACC',
 'GCTACCCTC',
 'CGTACCCTC',
 'TAGACCACC',
 'TAATCTTAG',
 'AGTACCCTC',
 'TAATTACCT',
 'CTAACCCTC',
 'AAGACCTTA',
 'TTTCCCACC',
 'GCTGTGATG',
 'ACTACCCTC',
 'GGCCACTCG',
 'GGCACCACC',
 'GGCCACGCG',
 'CGTGTGTTA',
 'TAATTACCG',
 'GGAACCACC',
 'TAATCTACC',
 'CGTTTCATC',
 'TAAACTACC',
 'TCCCACTTA',
 'GCCACCACC',
 'ATGACCACC',
 'TTTACCTTA',
 'GCGGTGATG',
 'AGTGTGCTC',
 'ATTACCATA',
 'GGCTTATGA',
 'CGTTTACCT',
 'ATTACCTGT',
 'ACTACCACC',
 'GGCACCTTA',
 'AGTTCTACC',
 'CGAACCCTC',
 'GTGGTGTTA',
 'AGTCACTTA',
 'TTTTGCAGC',
 'ATAACCACC',
 'TCTCCTCGG',
 'TCTTTCTAC',
 'GTAACCACC',
 'TCTTTTTAC',
 'ACTAAAATG',
 'TTTTTTTTA',
 'TTTGTCCGG',
 'CTGGTGTTA',
 'TCGTCTTAG',
 'AGTACCATA',
 'TCGACCACC',
 'TTTGCTATG',
 'TTTTTTATG',
 'CTGCACCTC',
 'CGTTTGTCA',
 'AGCAAACCT',
 'ATTGCTACC',
 'TCTAAAACC',
 'GCTGCTGTT',
 'GTAGTGTTA',
 'CGTGTGATG',
 'ACTACCTTA',
 'TTTGGGTGT',
 'GTTTTGCAG',
 'CAAACCCTC',
 'GTTTCACCT',
 'AAATCTCTC',
 'AGAACCCTC',
 'GCTTTCATC',
 'CTGT

In [78]:
k2p = fcgr.kmer2pixel
for i in top_n_kmers_GK:
    coords = k2p[i]
    print(avg_shap_values_all_GK[coords[0]-1][coords[1]-1])

0.06180052475001184
0.039144419556519254
0.023272544560151317
0.020149374913422273
0.016770722108532345
0.014885490219389345
0.012857712929727911
0.012443054926806965
0.009574424834914781
0.0083992927255778
0.008007047754408313
0.007661298791563443
0.007455956773852173
0.006931463698115792
0.00672256700907752
0.006477530967673614
0.006339671276559248
0.006313109587675468
0.005560736151277153
0.004672399679911247
0.004670192918311666
0.004203128963946409
0.004147185230744937
0.004132497071474558
0.003987085453038711
0.003902179491384535
0.003805698311174344
0.0037222840517905803
0.003499346870865475
0.0033844370945451366
0.003383123738889216
0.0031794529115157905
0.0031690654000327387
0.0031016007107123366
0.0030929833311191856
0.003048260552201244
0.00295200963104025
0.00293929339796327
0.0029139658929026703
0.0028846367614055015
0.002872792018082475
0.0028617101961378884
0.0028260164046889557
0.0027358580122919065
0.002538521600804662
0.002478081749581562
0.0024496674385242456
0.00244

In [79]:
tot = 0 
for i in top_n_kmers_GK:
    if i in clade_GK_marker_kmers:
        print(f"Kmer: {i}")
        tot += 1
print(tot)

Kmer: TTTACCCTC
Kmer: TCTACCCTC
Kmer: TTTCGTCCG
Kmer: TCTTTCTAC
Kmer: TCTTTTTAC
Kmer: TTTGTCCGG
Kmer: AGCAAACCT
Kmer: GTTTTGTCC
Kmer: AGGGTGTTA
Kmer: AAACCTTGT
10


In [80]:
# Aggiungiamo tutti i kmers spache spannano le marker mutations
k=9
clade_GK_marker_kmers1c = []
clade_GK_marker_kmers2c = []
clade_GK_marker_kmers3c = []
clade_GK_marker_kmers4c = []
clade_GK_marker_kmers1r = []
clade_GK_marker_kmers2r = []
clade_GK_marker_kmers3r = []
clade_GK_marker_kmers4r = []
clade_GK_marker_kmers1c.extend(get_kmers(clade_GK_ref[240-k+1:240+k]))
clade_GK_marker_kmers1r.extend(get_kmers(covid_reference[240-k+1:240+k]))

clade_GK_marker_kmers2c.extend(get_kmers(clade_GK_ref[3036-k+1:3036+k]))
clade_GK_marker_kmers2r.extend(get_kmers(covid_reference[3036-k+1:3036+k]))

clade_GK_marker_kmers3c.extend(get_kmers(clade_GK_ref[23402-k+1:23402+k]))
clade_GK_marker_kmers3r.extend(get_kmers(covid_reference[23402-k+1:23402+k]))

clade_GK_marker_kmers4c.extend(get_kmers(clade_GK_ref[22994-k+1:22994+k]))
clade_GK_marker_kmers4r.extend(get_kmers(covid_reference[22994-k+1:22994+k]))


for i in top_n_kmers_GK:
    if i in clade_GK_marker_kmers1c:
        print(f"Clade: {i} presente in Clade_GK_Marker_Kmers1c: {clade_GK_marker_kmers1c}")


for i in top_n_kmers_GK:
    if i in clade_GK_marker_kmers1r:
        print(f"Clade: {i} presente in Clade_GK_Marker_Kmers1r: {clade_GK_marker_kmers1r}")


for i in top_n_kmers_GK:
    if i in clade_GK_marker_kmers2c:
        print(f"Clade: {i} presente in Clade_GK_Marker_Kmers2c: {clade_GK_marker_kmers2c}")


for i in top_n_kmers_GK:
    if i in clade_GK_marker_kmers2r:
        print(f"Clade: {i} presente in Clade_GK_Marker_Kmers2r: {clade_GK_marker_kmers2r}")


for i in top_n_kmers_GK:
    if i in clade_GK_marker_kmers3c:
        print(f"Clade: {i} presente in Clade_GK_Marker_Kmers3c: {clade_GK_marker_kmers3c}")


for i in top_n_kmers_GK:
    if i in clade_GK_marker_kmers3r:
        print(f"Clade: {i} presente in Clade_GK_Marker_Kmers3r: {clade_GK_marker_kmers3r}")


for i in top_n_kmers_GK:
    if i in clade_GK_marker_kmers4c:
        print(f"Clade: {i} presente in Clade_GK_Marker_Kmers4c: {clade_GK_marker_kmers4c}")


for i in top_n_kmers_GK:
    if i in clade_GK_marker_kmers4r:
        print(f"Clade: {i} presente in Clade_GK_Marker_Kmers4r: {clade_GK_marker_kmers4r}")

Clade: TTTGTCCGG presente in Clade_GK_Marker_Kmers1c: ['CTAGGTTTT', 'TAGGTTTTG', 'AGGTTTTGT', 'GGTTTTGTC', 'GTTTTGTCC', 'TTTTGTCCG', 'TTTGTCCGG', 'TTGTCCGGG', 'TGTCCGGGT']
Clade: GTTTTGTCC presente in Clade_GK_Marker_Kmers1c: ['CTAGGTTTT', 'TAGGTTTTG', 'AGGTTTTGT', 'GGTTTTGTC', 'GTTTTGTCC', 'TTTTGTCCG', 'TTTGTCCGG', 'TTGTCCGGG', 'TGTCCGGGT']
Clade: TTTCGTCCG presente in Clade_GK_Marker_Kmers1r: ['CTAGGTTTC', 'TAGGTTTCG', 'AGGTTTCGT', 'GGTTTCGTC', 'GTTTCGTCC', 'TTTCGTCCG', 'TTCGTCCGG', 'TCGTCCGGG', 'CGTCCGGGT']
Clade: TTTACCCTC presente in Clade_GK_Marker_Kmers2c: ['TGTTCTTTT', 'GTTCTTTTT', 'TTCTTTTTA', 'TCTTTTTAC', 'CTTTTTACC', 'TTTTTACCC', 'TTTTACCCT', 'TTTACCCTC', 'TTACCCTCC']
Clade: TCTTTTTAC presente in Clade_GK_Marker_Kmers2c: ['TGTTCTTTT', 'GTTCTTTTT', 'TTCTTTTTA', 'TCTTTTTAC', 'CTTTTTACC', 'TTTTTACCC', 'TTTTACCCT', 'TTTACCCTC', 'TTACCCTCC']
Clade: TCTACCCTC presente in Clade_GK_Marker_Kmers2r: ['TGTTCTTTC', 'GTTCTTTCT', 'TTCTTTCTA', 'TCTTTCTAC', 'CTTTCTACC', 'TTTCTACCC', 'TTCTAC